# Boilerplate data preparation and computation

When detecting text-reuse with passim, one can start by identifying and eliminating the boilerplate to allow to remove superfluous data from the processing.

This notebook contains the code to perform various tasks relating to this:
- Preparing the input data directory, only containing data that should be part of boilerplate detection
- Light postprocessing of the boilerplate output and preparation of the actual text-reuse detection

### Imports

In [1]:
import os
import shutil
from tqdm import tqdm
import jq
import json
from typing import Any
from smart_open import open as smart_open_function
import jsonlines
import pickle

import dask.bag as db
import dask.dataframe as dd

from impresso_commons.utils.s3 import get_s3_resource

### Code

In [2]:
text_reuse_dir = '/scratch/piconti/impresso/text_reuse'
all_rebuilt_data_path = "rebuilt_data"
bp_rebuilt_data_path = "rebuilt_data_for_bp"

input_dir = os.path.join(text_reuse_dir, all_rebuilt_data_path)
output_dir = os.path.join(text_reuse_dir, bp_rebuilt_data_path)

## 1. Prepare the copied Rebuilt data for Boilerplate detection with passim

Before text-reuse detection with passim can be run, one must first run the tool in boilerplate mode to identify segments of text that should be excluded from the text-reuse search.

However not all the corpus should be confronted to boilerplate detection, and titles without any article-level segmentation should not be considered.

This small notebook aims to copy the wanted data (one that has article-level segmentation) to a new directory, where the files will be all together, as opposed to separated by title.

In [3]:
titles_no_bp = [
    "FedGazDe", "FedGazFr", "NZZ", "handelsztg", "arbeitgeber", "ACI", "AV", "Bombe", "Cancoire", "Castigat", "Charivari", "CharivariCH", "CL", 
    "Croquis", "EM", "esta", "FAM", "FAMDE", "FAN", "FAV1", "Fronde", "GAVi", "Grelot", "Griffe", "Guepe1851", "Guepe1887", 
    "JH", "JV", "JVE", "JY2", "MB", "ME", "MESSAGER", "Moniteur", "NS", "NV", "NV1", "NV2", "OBS", "ouistiti", "pages", "PAT", "PDL", "PJ", "PS", "RLA", "TouSuIl", "VVS", "VVS1"
]

In [4]:
copied, not_copied = [], []
for path, dir, files in os.walk(input_dir):
    if len(dir)==0:
        if path.split('/')[-1] in titles_no_bp:
            not_copied.extend(files)
            print(f"Not copying {path.split('/')[-1]} files since it has no article segmentation.")
        else:
            print(f"Copying {path.split('/')[-1]} files...")
            for file in tqdm(files):
                src_path = os.path.join(path, file)
                dest_path = os.path.join(output_dir, file)
                shutil.copy(src_path, dest_path)
                copied.append(file)

Not copying PS files since it has no article segmentation.
Copying lepji files...


100%|██████████| 37/37 [00:03<00:00, 11.11it/s]


Not copying NV files since it has no article segmentation.
Copying kommmit files...


100%|██████████| 1/1 [00:00<00:00, 19.84it/s]


Copying marieclaire files...


...

Copying SRT files...


100%|██████████| 1/1 [00:00<00:00, 62.10it/s]


Not copying handelsztg files since it has no article segmentation.
Copying GDL files...


100%|██████████| 197/197 [01:13<00:00,  2.68it/s]

Not copying Croquis files since it has no article segmentation.
Not copying esta files since it has no article segmentation.
Not copying FAMDE files since it has no article segmentation.


In [6]:
len(copied), len(not_copied), len(copied) + len(not_copied)

(3277, 991, 4268)

## 2. Upload the current boilerplate output to s3

For traceability, upload all the contents of the /out.json directory to S3 under a boilerplate partition

In [3]:
s3_staging_bucket = "41-processed-data-staging"
partition = "text-reuse/text-reuse_v1-0-0/boilerplate/out.json"
out_jsons_dir = os.path.join(text_reuse_dir, "passim_bp_output/out.json")

In [4]:
s3 = get_s3_resource()
bucket = s3.Bucket(s3_staging_bucket)

In [15]:
for filename in tqdm(os.listdir(out_jsons_dir)):
    if filename.endswith('json'):
        filepath = os.path.join(out_jsons_dir, filename)
        key_name = os.path.join(partition, filename)
        bucket.upload_file(filepath, key_name)
        #print("Uploaded %s to s3: %s", filepath, key_name)

  0%|          | 0/8942 [00:00<?, ?it/s]

100%|██████████| 8942/8942 [28:57<00:00,  5.15it/s]


## 3. Create the bp.plk file from the boilerplate output

#### First check the contents of some resulting jsons

In [3]:
out_jsons_dir = os.path.join(text_reuse_dir, "passim_bp_output/out.json")

os.listdir(out_jsons_dir)

['part-00570-58ebc338-4ee2-4196-96c3-c25f70a06e01-c000.json',
 '.part-04208-58ebc338-4ee2-4196-96c3-c25f70a06e01-c000.json.crc',
 '.part-01192-58ebc338-4ee2-4196-96c3-c25f70a06e01-c000.json.crc',
 'part-04374-58ebc338-4ee2-4196-96c3-c25f70a06e01-c000.json',
 '.part-01487-58ebc338-4ee2-4196-96c3-c25f70a06e01-c000.json.crc',
 'part-04250-58ebc338-4ee2-4196-96c3-c25f70a06e01-c000.json',
 '.part-00108-58ebc338-4ee2-4196-96c3-c25f70a06e01-c000.json.crc',
 '.part-03379-58ebc338-4ee2-4196-96c3-c25f70a06e01-c000.json.crc',
 '.part-03028-58ebc338-4ee2-4196-96c3-c25f70a06e01-c000.json.crc',
 '.part-00133-58ebc338-4ee2-4196-96c3-c25f70a06e01-c000.json.crc',
 '.part-00734-58ebc338-4ee2-4196-96c3-c25f70a06e01-c000.json.crc',
 '.part-02066-58ebc338-4ee2-4196-96c3-c25f70a06e01-c000.json.crc',
 'part-04418-58ebc338-4ee2-4196-96c3-c25f70a06e01-c000.json',
 '.part-02377-58ebc338-4ee2-4196-96c3-c25f70a06e01-c000.json.crc',
 '.part-02349-58ebc338-4ee2-4196-96c3-c25f70a06e01-c000.json.crc',
 '.part-01198-5

In [17]:
"part-00570-58ebc338-4ee2-4196-96c3-c25f70a06e01-c000.json" in os.listdir(out_jsons_dir)

True

In [4]:
def load_json(f_path: str) -> dict:
    lines = []
    with open(f_path, mode="r", encoding='utf-8') as f_in:
        for line in f_in:
            lines.append(json.loads(line))
    return lines

Aggregate all the outputs for BDC (since there's only one year) to see if heuristics can be concluded comparing them to the passim-rebuilt-no-bp contents

In [39]:
BDC_cis = []

In [41]:
for f in tqdm(os.listdir(out_jsons_dir)):
    added = False
    if f.endswith('json'):
        loaded = load_json(os.path.join(out_jsons_dir, f))
        for l in loaded:
            if 'BDC' in l['id']:
                BDC_cis.append(l)    
                added = True
    if added:
        print(f"BDC_cis now has {len(BDC_cis)} entries")
        #jq_value = jq.compile('.').input_value(loaded)
        #break

  0%|          | 39/8942 [00:05<31:50,  4.66it/s]

BDC_cis now has 1 entries


  1%|          | 47/8942 [00:06<26:02,  5.69it/s]

BDC_cis now has 2 entries


  1%|          | 110/8942 [00:15<35:07,  4.19it/s]

...

 99%|█████████▉| 8860/8942 [25:51<00:26,  3.15it/s]

BDC_cis now has 152 entries


100%|█████████▉| 8908/8942 [26:02<00:07,  4.83it/s]

BDC_cis now has 153 entries


100%|██████████| 8942/8942 [26:06<00:00,  5.71it/s]


In [42]:
BDC_cis

[{'id': 'BDC-1839-01-20-a-i0001',
  'cc': True,
  'date': '1839-01-20',
  'pages': [],
  'series': 'BDC',
  'text': 'AVIS\nDans le dessein d\'être utile au pays\net en même tems de satisfaire les vifs dé-\nsirs du public, le patriotisme de quelques\nCitoyens a pourvu à la publication d\'un\nbulletin des séances de la Constituante\nValaisanné. Les Citoyens qui d\'un mouve-\nment spontané ont généreusement contri-\nbué a couvrir les frais de cette publication\nespèrent que tous les vrais amis du pays\ns\'empresseront de s\'abonner k cette feuille\néminemment franche et patriotique, dont le\nbut est d\'éclairer le brave peuple Valai-\nsan sur des intérêts qui lui sont si chers\net de proclamer la vérité k là face du pays.\nDans cette feuille, résumé fidèle et im-\npartial des débats de la Constituante Va-\nlaisanne, le peuple apprendra de quelle\nanière sont traités\'ses intérêts.\nL\'Assemblée Constituante a voté par\nElle a de plus décidé, encore par accla-\nmation, que des places serai

In [53]:
filepath = '/scratch/piconti/impresso/text_reuse/BDC_bp_cis_example.json'
with smart_open_function(filepath, "ab") as fout:
    writer = jsonlines.Writer(fout)

    # items = [issue.issue_data for issue in issues]
    writer.write_all(BDC_cis)

Repeat approach for Gazgardlux because BDC has too many text reusie with l'echo des alpes (EDA)

In [5]:
gaz_cis = []

In [7]:
for f in tqdm(os.listdir(out_jsons_dir)):
    added = False
    if f.endswith('json'):
        loaded = load_json(os.path.join(out_jsons_dir, f))
        for l in loaded:
            if 'gazgrdlux' in l['id']:
                gaz_cis.append(l)    
                added = True
    if added:
        print(f"gaz_cis now has {len(gaz_cis)} entries")
        #jq_value = jq.compile('.').input_value(loaded)
        #break

  0%|          | 1/8942 [00:00<45:27,  3.28it/s]

gaz_cis now has 105 entries


  0%|          | 18/8942 [00:01<08:04, 18.43it/s]

gaz_cis now has 110 entries


  0%|          | 20/8942 [00:01<10:38, 13.98it/s]

...

gaz_cis now has 5246 entries


100%|█████████▉| 8933/8942 [24:16<00:00, 10.46it/s]

gaz_cis now has 5248 entries


100%|██████████| 8942/8942 [24:16<00:00,  6.14it/s]

gaz_cis now has 5249 entries


In [8]:
filepath = '/scratch/piconti/impresso/text_reuse/gazgrdlux_bp_cis_example.json'
with smart_open_function(filepath, "ab") as fout:
    writer = jsonlines.Writer(fout)

    # items = [issue.issue_data for issue in issues]
    writer.write_all(gaz_cis)

### Actually create the pb.pkl dataframe

From looking at examples the following heuristics have been devised:
- Only entries with the field `"src"` are actually boilerplate.
  - In the format: `{"id": "BDC-1839-03-18-a-i0005_1658_1952", "src": {"id": "BDC-1839-03-16-a-i0004", "start": [...]`
- For each entry with the field `"src"`:
  - Two text passages are linked: the value of `id` and the value of `src.id`
  - The value of `id` will often have additional values appended after the usual CI id ("_1658_1952" here). These should be removed
  - Both ids (fields `id` and `src.id`) should be included in the bp.pkl output as separate rows
- All rows should also have a column `"is_boilerplate"` set to `True`.

In [92]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [93]:
out_jsons_dir = os.path.join(text_reuse_dir, "passim_bp_output/out.json")
pb_pkl_out_filepath = os.path.join(text_reuse_dir, "pb.pkl")

json_parts = list(map(lambda f: os.path.join(out_jsons_dir, f), filter(lambda f: f.endswith('json'), os.listdir(out_jsons_dir))))

json_file_chunks = chunks(json_parts, 5)

In [94]:
def format(entries: list[dict[str,Any]]) -> list[dict[str, str | bool]]:
    out = []
    for e in entries:
        e_id = e['id'].split('_')[0]
        # append first id
        out.append({
            'id': e_id, 
            "is_boilerpate": True,
        })
        # add second id
        out.append({
            "id": e['src']['id'],
            "is_boilerplate": True
        })

    return out

In [95]:
# The output df starts empty
pb_df = None

for chunk in tqdm(json_file_chunks):
    # read the chunk of 5 parts into a dask bag
    bp_bag = db.read_text(chunk).map(json.loads)
    # filter out all non-boilerplate ids: only keept entries where the field "src" is defined
    filtered = bp_bag.filter(lambda x: 'src' in x)

    formatted = filtered.map_partitions(format)

    chunck_df = formatted.to_dataframe(meta={"id": str, "is_boilerplate": bool}).persist()

    print(f"Appending new data to pb.pkl: \n{chunck_df.tail()}")

    if pb_df is not None:
        pb_df = dd.concat([pb_df, chunck_df])
    else:
        pb_df = chunck_df

    print(f"New lenght of the dataframe: {len(pb_df)}")

# writing df to pickle file
with open(pb_pkl_out_filepath, 'wb') as handle:
    pickle.dump(pb_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(pb_pkl_out_filepath, 'rb') as handle:
    b = pickle.load(handle)

print(all(pb_df == b))


1it [00:02,  2.30s/it]

Appending new data to pb.pkl: 
                                    id  is_boilerplate
4847      tageblatt-1946-02-18-a-i0042            True
4848      tageblatt-1947-04-12-a-i0164            True
4849      tageblatt-1947-04-04-a-i0095            True
4850  waechtersauer-1862-03-01-a-i0016            True
4851  waechtersauer-1862-02-22-a-i0013            True
New lenght of the dataframe: 23606


2it [00:04,  2.19s/it]

Appending new data to pb.pkl: 
                                id  is_boilerplate
4591  tageblatt-1937-09-30-a-i0112            True
4592  tageblatt-1942-07-07-a-i0063            True
4593  tageblatt-1942-06-30-a-i0055            True
4594  tageblatt-1947-03-14-a-i0086            True
4595  tageblatt-1947-03-12-a-i0083            True
New lenght of the dataframe: 47284


3it [00:06,  2.15s/it]

...

New lenght of the dataframe: 5399422
Appending new data to pb.pkl: 
                                   id  is_boilerplate
4767     tageblatt-1948-12-20-a-i0066            True
4768     tageblatt-1950-08-14-a-i0102            True
4769     tageblatt-1950-08-12-a-i0169            True
4770  volkfreu1869-1875-01-17-a-i0017            True
4771  volkfreu1869-1875-01-10-a-i0029            True


233it [22:27, 12.29s/it]

New lenght of the dataframe: 5422592
Appending new data to pb.pkl: 
                                id  is_boilerplate
4553  tageblatt-1948-02-10-a-i0077            True
4554  tageblatt-1949-04-16-a-i0147            True
4555  tageblatt-1949-04-13-a-i0097            True
4556  tageblatt-1949-12-31-a-i0180            True
4557  tageblatt-1949-12-15-a-i0105            True


233it [22:35,  5.82s/it]


KeyboardInterrupt: 

In [47]:
23606/79225

0.2979615020511202

In [5]:
# final lenght of the pb.pkl file
final_len = 20814910
total_num_cis = 58619584

# proportion of boilerplate in CIs
final_len/total_num_cis

0.35508457378339636

#### Upload the resulting file to S3

In [8]:
pb_pkl_out_filepath = os.path.join(text_reuse_dir, "pb.pkl")
s3_staging_bucket = "41-processed-data-staging"
pb_partition = "text-reuse/text-reuse_v1-0-0/boilerplate/"

s3 = get_s3_resource()
bucket = s3.Bucket(s3_staging_bucket)

In [10]:
pb_pkl_out_filepath = os.path.join(text_reuse_dir, "pb.pkl")
bp_key_name = os.path.join(pb_partition, "pb.pkl")
bucket.upload_file(pb_pkl_out_filepath, bp_key_name)